In [1]:
import numpy as np
import torch
from torchvision.io import VideoReader
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
import math
import sys
import pickle
import os

In [2]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
    return result


def find_nearest(array, value):
    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return idx-1
    
    else:
        return idx



In [6]:
class FrameWithAngle(object):
    def __init__(self, frame, angle):
        self.frame = frame
        self.angle = angle


# Pre process data

base_dir = "/media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1"
save_dir = "/media/nevin/Trash Games1/Downloads/comma2k19/processed"

dirs = os.listdir(base_dir)

for _dir in dirs:
    project_dir = os.path.join(base_dir, _dir)
    segments = os.listdir(project_dir)

    videos = []

    for segment in segments:
        seg_dir = os.path.join(project_dir, segment)
        video_path = os.path.join(seg_dir, "video.hevc")

        frame_times = np.load(os.path.join(seg_dir, "global_pose/", "frame_times"))
        angle_t = np.load(os.path.join(seg_dir, "processed_log/", "CAN/", "steering_angle/", "t"))
        angle_v = np.load(os.path.join(seg_dir, "processed_log/", "CAN/", "steering_angle/", "value"))

        video = cv.VideoCapture(video_path)

        frames = []

        # print(f"reading {video_path}")

        if video.isOpened() == False:
            print(f"Failed to open video {video_path}")
            continue

        i = 0

        while video.isOpened():
            status, frame = video.read()

            if status == False:
                break

            # find the nearest frame which matches the angle input
            rot_idx = find_nearest(angle_t, frame_times[i])
            angle = angle_v[rot_idx]

            frames.append(FrameWithAngle(frame, angle))
            i += 1
        
        # print(frames)
        # print(sys.getsizeof(frames))

    videos = frames
        # videos.append(frames)
    

    with open(os.path.join(save_dir, f"{_dir}.pkl"), "wb") as f:
        pickle.dump(videos, f)
        print(f"dumped video for file {f.name}")
    # pickle.dumps()

    # print(files)

# print(dirs)

dumped video for file /media/nevin/Trash Games1/Downloads/comma2k19/processed/b0c9d2329ad1606b_2018-07-27--06-03-57.pkl
dumped video for file /media/nevin/Trash Games1/Downloads/comma2k19/processed/b0c9d2329ad1606b_2018-07-27--06-50-48.pkl


KeyboardInterrupt: 

In [14]:
class VideoAngleDataset(torch.utils.data.IterableDataset):
    def __init__(self, videos):
        super(VideoAngleDataset).__init__()
        self.videos = videos
        self.video = None
    

    def __iter__(self):
        self.vid_idx = 0
        self.current_vid = None
        self.frame_idx = 0
        return self
    

    def __next__(self):
        if self.vid_idx >= len(self.videos):
            raise StopIteration

        if self.current_vid == None:
            self.current_vid = iter(self.videos[self.vid_idx])
        
        frame = next(self.current_vid, None)
        if frame == None:
            self.vid_idx += 1

            if self.vid_idx >= len(self.videos):
                raise StopIteration

            self.current_vid = iter(self.videos[self.vid_idx])

        return frame

In [10]:
class VideoWithInfo:
    def __init__(self, vid_path, ft_path, angle_t_path, angle_v_path):
        self.vid_path = vid_path
        self.ft_path = ft_path
        self.angle_t_path = angle_t_path
        self.angle_v_path = angle_v_path
        self.video = None
    

    def __iter__(self):
        self.video = cv.VideoCapture(self.vid_path)
        self.frame_times = np.load(self.ft_path)
        self.angle_t = np.load(self.angle_t_path)
        self.angle_v = np.load(self.angle_v_path)
        self.frame_idx = 0

        return self


    def __next__(self):
        if self.video.isOpened() == False:
            raise StopIteration
        
        status, frame = self.video.read()

        if status == False:
            raise StopIteration

        rot_idx = find_nearest(self.angle_t, self.frame_times[self.frame_idx])
        angle = self.angle_v[rot_idx]

        self.frame_idx += 1
        return {'frame': frame, 'angle': angle}



In [15]:
base_dir = "/media/nevin/Trash Games1/Downloads/comma2k19/extracted/Chunk_1/b0c9d2329ad1606b_2018-07-27--06-03-57"
wheel = cv.imread("/home/nevin/Downloads/wheel.png")
video_1 = VideoWithInfo(f"{base_dir}/3/video.hevc", f"{base_dir}/3/global_pose/frame_times", f"{base_dir}/3/processed_log/CAN/steering_angle/t", f"{base_dir}/3/processed_log/CAN/steering_angle/value")
video_2 = VideoWithInfo(f"{base_dir}/4/video.hevc", f"{base_dir}/4/global_pose/frame_times", f"{base_dir}/4/processed_log/CAN/steering_angle/t", f"{base_dir}/4/processed_log/CAN/steering_angle/value")

videos = [video_1, video_2]
dataset = VideoAngleDataset(videos)

for frame in dataset:
    pass
    # print(frame["frame"].shape)
    # rotated = rotate_image(wheel, frame["angle"])
    # cv.imshow("frame", frame["frame"])
    # cv.imshow("wheel", rotated)

    # key = cv.waitKey()

    # if key == 113:
    #     cv.destroyAllWindows()
    #     break

    # print(frame["angle"])
    # print(frame[0])

# cv.destroyAllWindows()
# print(sys.getsizeof(video))

<VideoCapture 0x7fe3dd1d79b0>
frame is none
<VideoCapture 0x7fe302691710>
frame is none
stopping iteration


In [8]:
with open(os.path.join(save_dir, "b0c9d2329ad1606b_2018-07-27--06-03-57.pkl"), "rb") as f:
    data = pickle.load(f)

    print(data)

[<__main__.FrameWithAngle object at 0x7f2e8f7a7be0>, <__main__.FrameWithAngle object at 0x7f2e8f647610>, <__main__.FrameWithAngle object at 0x7f2de45d0c10>, <__main__.FrameWithAngle object at 0x7f2de45d0a60>, <__main__.FrameWithAngle object at 0x7f2de45d0880>, <__main__.FrameWithAngle object at 0x7f2dc42b59d0>, <__main__.FrameWithAngle object at 0x7f2de8618a00>, <__main__.FrameWithAngle object at 0x7f2de8618f10>, <__main__.FrameWithAngle object at 0x7f2de8618ca0>, <__main__.FrameWithAngle object at 0x7f2de8618910>, <__main__.FrameWithAngle object at 0x7f2de8618c70>, <__main__.FrameWithAngle object at 0x7f2de8618e80>, <__main__.FrameWithAngle object at 0x7f2de86187c0>, <__main__.FrameWithAngle object at 0x7f2de45c3eb0>, <__main__.FrameWithAngle object at 0x7f2de45c3d90>, <__main__.FrameWithAngle object at 0x7f2de45c3700>, <__main__.FrameWithAngle object at 0x7f2de45c31f0>, <__main__.FrameWithAngle object at 0x7f2de45c3d30>, <__main__.FrameWithAngle object at 0x7f2de45c3f70>, <__main__.F